In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

soup = BeautifulSoup(requests.get(f"https://books.toscrape.com/catalogue/category/books/travel_2/index.html").content,"lxml")

section_dicto = {}
for i in range(1,51):
    sections = soup.find('ul',{"class":"nav nav-list"}).find_all("a")[i].get_text().strip()
    section_dicto[sections] = sections.lower()+f"_{i+1}"

for k, v in section_dicto.items():
    if ' ' in v:
        section_dicto[k] = v.replace(' ','-')

the_section = input("Enter the section : ")

soup = BeautifulSoup(requests.get(f"http://books.toscrape.com/catalogue/category/books/{section_dicto[the_section.title()]}/index.html").content,"lxml")

rate_dict = {"One":"1 star","Two":"2 stars","Three":"3 stars","Four":"4 stars","Five":"5 stars"}

with open("books.csv","w",newline='',encoding="UTF-8") as fd:

    heads = ["Title","Rate","Price"]
    writer = csv.DictWriter(fd,heads)
    writer.writeheader()

    num_of_pages = soup.find("li",{"class":"current"})
    
    if num_of_pages == None:
        num_of_pages = 1
    else:
        num_of_pages = soup.find("li",{"class":"current"}).get_text().split()[-1]

    for page in range(1,int(num_of_pages)+1):

        if num_of_pages == 1:
            url = f"http://books.toscrape.com/catalogue/category/books/{section_dicto[the_section.title()]}/index.html"
        else:
            url = f'http://books.toscrape.com/catalogue/category/books/{section_dicto[the_section.title()]}/page-{str(page)}.html'

        resp = requests.get(url)
        soup = BeautifulSoup(resp.content,"lxml")

        all_articles = soup.find_all("article",{"class":"product_pod"})

        for article in all_articles:
            title = article.find("h3").find("a").attrs["title"]
            price = article.find("p",{"class":"price_color"}).get_text()
            rate = rate_dict[article.find("p",{"class":"star-rating"}).attrs["class"][1]]

            writer.writerow({"Title":title,"Rate":rate,"Price":price})


In [ ]:
print("The Sections : \n")
for k in section_dicto.keys():
    print(k)

The Sections : 

Travel
Mystery
Historical Fiction
Sequential Art
Classics
Philosophy
Romance
Womens Fiction
Fiction
Childrens
Religion
Nonfiction
Music
Default
Science Fiction
Sports and Games
Add a comment
Fantasy
New Adult
Young Adult
Science
Poetry
Paranormal
Art
Psychology
Autobiography
Parenting
Adult Fiction
Humor
Horror
History
Food and Drink
Christian Fiction
Business
Biography
Thriller
Contemporary
Spirituality
Academic
Self Help
Historical
Christian
Suspense
Short Stories
Novels
Health
Politics
Cultural
Erotica
Crime


In [1]:
data = pd.read_csv("books.csv")

data